In [1]:
import h5py
import numpy as np
import tensorflow as tf
import argparse
import os
import random as rn
import matplotlib.pyplot as plt
import time

2022-09-28 19:24:47.193306: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-28 19:24:47.279085: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-28 19:24:47.608145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-28 19:24:47.608180: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

Helper Functions

In [2]:
def use_gpu(Flag):
# (0 = GPU, -1 = CPU)    
    os.environ['CUDA_VISIBLE_DEVICES'] = Flag  
    if tf.test.gpu_device_name():
        print('GPU found')
    else:
        print("No GPU found")
    return


## The only 2 choices to make in the notebook ! 

In [16]:
n = 7 # (Number of quantization bits)
model_choice = 'FC' # (Choose between 'CNN' or 'FC')

if model_choice =='FC' and n==7:
    print('YOU CAN`T ! This make the input size to big for HLS4ML (>4096)')

YOU CAN`T ! This make the input size to big for HLS4ML (>4096)


Setup

In [4]:
if model_choice =='CNN':
    gpu = '0' # ('0' = GPU, '-1' = CPU)       
elif model_choice =='FC':
    gpu = '-1' # ('0' = GPU, '-1' = CPU)       
Qtype = 'opt'   # ('opt'= optimized quantization)

# Seed Setup
seed_value = 1
keras_kernel_initializer = 'lecun_uniform'
os.environ['PYTHONHASHSEED'] = str(seed_value)
tf.keras.utils.set_random_seed(seed_value)

# Selecting CPU or GPU     
use_gpu(gpu)    

# Vivado stuff
os.environ['PATH'] = '/opt/Xilinx/Vivado/2019.1/bin:' + os.environ['PATH']
# os.environ['XILINXD_LICENSE_FILE'] = '6062@xilinx.gel.usherbrooke.ca:16062@xilinx.gel.usherbrooke.ca'
os.environ['XILINXD_LICENSE_FILE'] = 'Xilinx.gegi.usherbrooke.ca:16062@xilinx.gegi.usherbrooke.ca'


No GPU found


2022-09-28 19:24:48.807464: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-28 19:24:48.845899: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-28 19:24:48.845980: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: thebeast
2022-09-28 19:24:48.846000: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: thebeast
2022-09-28 19:24:48.846172: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.43.4
2022-09-28 19:24:48.846245: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.4

Loading Dataset

In [5]:
# test set
X_test = np.load('./dataset/test_set_' + str(n) + 'bits_' + Qtype + '.npz')['X_test']
Y_test = np.load('./dataset/test_set_' + str(n) + 'bits_' + Qtype + '.npz')['Y_test']

print(X_test.shape)
print(Y_test.shape)

(100000, 16, 128, 1)
(100000,)


Loading model (please indicate if you want the CNN or the FC)

In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects
co = {}
_add_supported_quantized_objects(co)


if model_choice =='CNN':
    qmodel_pruned = load_model("./preTrained_models/CNN_quantized_pruned_model_pulses_" + str(n) + "bits_" + Qtype + ".h5", custom_objects=co)
elif model_choice =='FC':
    qmodel_pruned = load_model("./preTrained_models/FC_quantized_pruned_model_pulses_" + str(n) + "bits_" + Qtype + ".h5", custom_objects=co)

qmodel_pruned.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics='accuracy')
qmodel_pruned.summary()

Model: "fc"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 fc1 (QDense)                (None, 4)                 8196      
                                                                 
 relu1 (QActivation)         (None, 4)                 0         
                                                                 
 fc2 (QDense)                (None, 32)                160       
                                                                 
 relu2 (QActivation)         (None, 32)                0         
                                                                 
 fc3 (QDense)                (None, 32)                1056      
                                                                 
 relu3 (QActivation)         (None, 32)                0        

/home/goub2904/git/cookiebox_hsl4ml/CB_HLS4ML/venv/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Testing the pre-trained model 

In [7]:
_,test_acc_baseline = qmodel_pruned.evaluate(X_test, Y_test, verbose=False)
print(test_acc_baseline)

0.8168100118637085


HLS4ML Export

In [8]:
model2hls = qmodel_pruned

for layer in model2hls.layers:
    if layer.__class__.__name__ in ['QConv2D', 'QDense']:
        w = layer.get_weights()[0]
        layersize = np.prod(w.shape)
        print("{}: {}".format(layer.name,layersize)) # 0 = weights, 1 = biases
        if (layersize > 4096): # assuming that shape[0] is batch, i.e., 'None'
            print("Layer {} is too large ({}), are you sure you want to train?".format(layer.name,layersize))

fc1: 8192
Layer fc1 is too large (8192), are you sure you want to train?
fc2: 128
fc3: 1024
output: 160


In [9]:
import hls4ml

hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

config = hls4ml.utils.config_from_keras_model(model2hls, granularity='name')
config['LayerName']['output_softmax']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['output_softmax']['inv_table_t'] = 'ap_fixed<18,4>'


config['Model']['Strategy'] = 'Latency'
config['Model']['ReuseFactor'] = 1


if model_choice =='CNN':
    output_dir = 'TO_FPGA_' + str(n) + 'bits_' + Qtype + '/hls4ml_prj_FPGA_CNN'
    io_type = 'io_stream'
elif model_choice =='FC':
    output_dir = 'TO_FPGA_' + str(n) + 'bits_' + Qtype + '/hls4ml_prj_FPGA_FC'
    io_type = 'io_parallel'
                           

hls_model = hls4ml.converters.convert_from_keras_model(model2hls,
                                                        hls_config=config,
                                                        output_dir=output_dir,
                                                        backend='VivadoAccelerator',
                                                        io_type=io_type,
                                                        board='zcu102',
                                                        part='xczu9eg-ffvb1156-2-e')

hls_model.compile()                                                       

/home/goub2904/git/cookiebox_hsl4ml/CB_HLS4ML/venv/lib/python3.10/site-packages/hls4ml/converters/__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


Interpreting Sequential
Topology:
Layer name: input_2, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: fc2, layer type: QDense
Layer name: relu2, layer type: QActivation
Layer name: fc3, layer type: QDense
Layer name: relu3, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: output_softmax, layer type: Activation
Interpreting Sequential
Topology:
Layer name: input_2, layer type: InputLayer, input shapes: [[None, 16, 128, 1]], output shape: [None, 16, 128, 1]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 16, 128, 1]], output shape: [None, 2048]
Layer name: fc1, layer type: QDense, input shapes: [[None, 2048]], output shape: [None, 4]
Layer name: relu1, layer type: Activation, input shapes: [[None, 4]], output shape: [None, 4]
Layer name: fc2, layer type: QDense, input shapes: [[None, 4]], output shape: [None, 32]
Layer name: relu2, layer type: Activation, input shapes: [[None, 32]], ou

HLS4ML Dataset (HLS4ML inference are much longer to do than with keras when using CPU. For that reason we make a subset that we will also use to transfer and test on the FPGA)

In [10]:
X_test_subset = X_test[0:2**14]
Y_test_subset = Y_test[0:2**14]

np.savez('./dataset/subset_' + str(n) + 'bits_' + Qtype + '.npz',X_test_subset=X_test_subset,Y_test_subset=Y_test_subset)

X_test_subset = np.load('./dataset/subset_' + str(n) + 'bits_' + Qtype + '.npz')['X_test_subset']
Y_test_subset = np.load('./dataset/subset_' + str(n) + 'bits_' + Qtype + '.npz')['Y_test_subset']


print(X_test_subset.shape)
print(Y_test_subset.shape)

(16384, 16, 128, 1)
(16384,)


HLS4ML model perfo

In [11]:
from sklearn.metrics import accuracy_score

predict_hls = hls_model.predict(np.ascontiguousarray(X_test_subset))

print('HLS4ML accuracy = %.4f' % (accuracy_score(Y_test_subset, np.argmax(predict_hls,axis=1))))

HLS4ML accuracy = 0.5886


Synthesize (Now synthesize the model, and also export the IP.)

In [30]:
hls_model.build(csim=False, export=True)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'goub2904' on host 'thebeast' (Linux_x86_64 version 5.15.0-46-generic) on Mon Sep 26 18:15:38 EDT 2022
INFO: [HLS 200-10] On os Ubuntu 22.04 LTS
INFO: [HLS 200-10] In directory '/home/goub2904/git/cookiebox_hsl4ml/CB_HLS4ML/TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Creating and opening project '/home/goub2904/git/cookiebox_hsl4ml/CB_HLS4ML/TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproj

{'EstimatedClockPeriod': '4.349',
 'BestLatency': '2049',
 'WorstLatency': '2049',
 'IntervalMin': '2050',
 'IntervalMax': '2050',
 'BRAM_18K': '96',
 'DSP48E': '342',
 'FF': '39704',
 'LUT': '104022',
 'URAM': '0',
 'AvailableBRAM_18K': '1824',
 'AvailableDSP48E': '2520',
 'AvailableFF': '548160',
 'AvailableLUT': '274080',
 'AvailableURAM': '0'}

In [31]:
hls4ml.report.read_vivado_report(output_dir)

Found 1 solution(s) in TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject_axi'
* Date:           Mon Sep 26 18:20:27 2022

* Version:        2019.1 (Build 2552052 on Fri May 24 15:28:33 MDT 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: zynquplus
* Target device:  xczu9eg-ffvb1156-2-e


== Performance Estimates
+ Timing (ns): 
    * Summary: 
    +--------+-------+----------+------------+
    |  Clock | Target| Estimated| Uncertainty|
    +--------+-------+----------+------------+
    |ap_clk  |   5.00|     4.349|        0.62|
    +--------+-------+----------+------------+

+ Latency (clock cycles): 
    * Summary: 
    +------+------+------+------+----------+
    |   Latency   |   Interval  | Pipeline |
    |  min |  max |  min |  max |   Type   |
    +------+------+------+------+----------+
    |  2049|  2049|  2050|  2050| dat

What is under have not been modify to support 3,5 and 7 bits quantization

Make bitfile

In [ ]:
# hls4ml.templates.VivadoAcceleratorBackend.make_bitfile(hls_model)

In [ ]:
# if model_choice =='CNN':
#     !sed -n '30,45p' TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper_utilization_placed.rpt
# elif model_choice =='FC':
#     !sed -n '30,45p' TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper_utilization_placed.rpt

Exporting to FPGA (the path have to be hardcoded sorry for that)

In [ ]:
# if model_choice =='CNN':
#     !mkdir TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/package/
#     !cp TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper.bit TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/package/hls4ml_nn.bit
#     !cp TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/myproject_vivado_accelerator/project_1.srcs/sources_1/bd/design_1/hw_handoff/design_1.hwh TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/package/hls4ml_nn.hwh
#     !cp TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/axi_stream_driver.py TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/package/
#     !cp X_test_FPGA_5bits_opt.npy Y_test_FPGA_5bits_opt.npy TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/package
#     !tar -czvf TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/package.tar.gz -C TO_FPGA_5bits_opt/hls4ml_prj_FPGA_CNN/package/ .
# elif model_choice =='FC':
#     !mkdir TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/package/
#     !cp TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper.bit TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/package/hls4ml_nn.bit
#     !cp TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/myproject_vivado_accelerator/project_1.srcs/sources_1/bd/design_1/hw_handoff/design_1.hwh TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/package/hls4ml_nn.hwh
#     !cp TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/axi_stream_driver.py TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/package/
#     !cp X_test_FPGA_5bits_opt.npy Y_test_FPGA_5bits_opt.npy TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/package
#     !tar -czvf TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/package.tar.gz -C TO_FPGA_5bits_opt/hls4ml_prj_FPGA_FC/package/ .